# Projekt
**Filip Jakovina, 0036535147**
## A Machine Learning Strategy For Predicting March Madness Winnerss

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Priprema i vizualizacija podataka

## Učitavanje podataka

In [45]:
rscr = pd.read_csv("data/regular_season_compact_results.csv")
rsdr = pd.read_csv("data/regular_season_detailed_results.csv")
s = pd.read_csv("data/seasons.csv")
t = pd.read_csv("data/teams.csv")
tcr = pd.read_csv("data/tourney_compact_results.csv")
tdr = pd.read_csv("data/tourney_detailed_results.csv")
tse = pd.read_csv("data/tourney_seeds.csv")
tsl = pd.read_csv("data/tourney_slots.csv")

## Prikaz i analiza podataka

### Sezone
Popis sezona

In [46]:
with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    display(s)

,season,dayzero,regionW,regionX,regionY,regionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/02/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast
5,1990,10/30/1989,East,Midwest,Southeast,West
6,1991,10/29/1990,East,Southeast,Midwest,West
7,1992,11/04/1991,East,West,Midwest,Southeast
8,1993,11/02/1992,East,Midwest,Southeast,West
9,1994,11/01/1993,East,Southeast,Midwest,West


Pojašnjenja značajki:
- Season &ndash; jedno-znakovni identifikator sezone
- Years &ndash; godine kroz koje se proteže sezona
- Dayzero &ndash; datum početka sezone
- RegionW/X/Y/Z &ndash; 4 regije u završnom turniru sezone

In [47]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   season   31 non-null     int64 
 1   dayzero  31 non-null     object
 2   regionW  31 non-null     object
 3   regionX  31 non-null     object
 4   regionY  31 non-null     object
 5   regionZ  31 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.6+ KB


In [48]:
s.describe(include='all')

,season,dayzero,regionW,regionX,regionY,regionZ
count,31.000000,31,31,31,31,31
unique,NaN,31,3,7,6,7
top,NaN,10/29/1984,East,Midwest,Midwest,West
freq,NaN,1,28,9,18,21
mean,2000.000000,NaN,NaN,NaN,NaN,NaN
std,9.092121,NaN,NaN,NaN,NaN,NaN
min,1985.000000,NaN,NaN,NaN,NaN,NaN
25%,1992.500000,NaN,NaN,NaN,NaN,NaN
50%,2000.000000,NaN,NaN,NaN,NaN,NaN
75%,2007.500000,NaN,NaN,NaN,NaN,NaN


Ova tablica ne sadrži nedostajuće i stršeće podatke. <br>
Sljedeće ćemo vidjeti koje regije se koliko puta pojavljuju:

In [49]:
columns = ['regionW', 'regionX', 'regionY', 'regionZ']
region_counts = {}

for column in columns:
    counts = s[column].value_counts().to_dict()
    for region, count in counts.items():
        region_counts[region] = region_counts.get(region, 0) + count

region_counts = sorted(region_counts.items(), key=lambda item: item[1], reverse=True)
for region, count in region_counts:
    print(f"{region}: {count}")

East: 28
West: 28
Midwest: 27
South: 14
Southeast: 14
Atlanta: 2
Albuquerque: 1
Phoenix: 1
Chicago: 1
Oakland: 1
EastRutherford: 1
Austin: 1
Minneapolis: 1
StLouis: 1
Syracuse: 1
WashingtonDC: 1
Southwest: 1


Iz sljedećeg prikaza možemo razlikovati dvije vrste regija: Geografski određene regije(East, West, Midwest, South, Southeast, Southwest)
koje se pojavljuju više puta <br>te regije koje su gradovi(Atlanta, Albuquerque, Phoenix, Chicago...)<br>
Imena tih gradova valjalo bi zamijeniti sa geografskom regijom kojoj ti gradovi pripadaju i ponoviti brojanje.

In [50]:
replacements = {
    'Atlanta': 'South',
    'Albuquerque': 'West',
    'Phoenix': 'West',
    'Chicago': 'Midwest',
    'Oakland': 'West',
    'EastRutherford': 'East',
    'Austin': 'South',
    'Minneapolis': 'West',
    'StLouis': 'Midwest',
    'Syracuse': 'East',
    'WashingtonDC': 'East',
}
for column in columns:
    s[column] = s[column].replace(replacements)

region_counts = {}

for column in columns:
    counts = s[column].value_counts().to_dict()
    for region, count in counts.items():
        region_counts[region] = region_counts.get(region, 0) + count

region_counts = sorted(region_counts.items(), key=lambda item: item[1], reverse=True)
for region, count in region_counts:
    print(f"{region}: {count}")

West: 32
East: 31
Midwest: 29
South: 17
Southeast: 14
Southwest: 1


### Timovi
Popis identifikatora i imena timova

In [51]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(t)

,team_id,team_name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M
...,...,...
359,1460,Wright St
360,1461,Wyoming
361,1462,Xavier
362,1463,Yale


In [52]:
t.describe(include='all')

,team_id,team_name
count,364.000000,364
unique,NaN,364
top,NaN,Abilene Chr
freq,NaN,1
mean,1282.500000,NaN
std,105.221988,NaN
min,1101.000000,NaN
25%,1191.750000,NaN
50%,1282.500000,NaN
75%,1373.250000,NaN


Kako ova tablica sadrži samo imena timova nije potrebno tražiti nedostajuće i stršeće vrijednosti te raditi vizualizacije

### Rezultati regularnog dijela sezone
Rezultati svake utakmice u regularnom dijelu sezone, uz potpunu statistiku od sezone 2003. do 2014.

In [53]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(rsdr)

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,wfgm3,wfga3,wftm,wfta,wor,wdr,wast,wto,wstl,wblk,wpf,lfgm,lfga,lfgm3,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60513,2014,132,1196,61,1246,60,N,0,23,53,8,19,7,17,13,25,12,13,9,3,22,18,51,3,11,21,26,12,24,9,12,4,8,19
60514,2014,132,1277,69,1276,55,N,0,27,54,2,17,13,16,14,24,14,10,7,6,19,17,54,6,23,15,16,13,15,7,9,3,3,15
60515,2014,132,1386,65,1433,61,N,0,24,50,6,17,11,20,7,33,13,12,2,5,13,24,65,5,19,8,15,13,25,9,5,6,6,18
60516,2014,132,1418,82,1209,81,N,1,32,75,11,26,7,14,20,26,15,8,4,1,15,28,58,5,19,20,21,4,26,8,7,5,7,15


Pojašnjenja značajki (W - pobjednik, L - gubitnik):
 - Season &ndash; identifikator sezone (odgovara stupcu "season" u datoteci "seasons.csv")
 - Daynum &ndash; broj dana nakon "dayzero" datuma u "seasons.csv"
 - Wteam/Lteam &ndash; Id momčadi iz "teams.csv" 
 - Wscore/Lscore &ndash; Broj poena
 - Wloc &ndash; Lokacija pobjedničke momčadi (H - domaćin, A - gost, N - neutralan teren)
 - Numot &ndash; Broj odigranih produžetaka na utakmici
 - Wfgm/Lfgm &ndash; Broj pogođenih koševa (2 poena)
 - Wfga/Lfga &ndash; Broj bacanja prema košu (2 poena) 
 - Wfgm3/Lfgm3 &ndash; Broj pogođenih trica (3 poena)
 - Wfga3/Lfga3 &ndash; Broj bacanja trica (3 poena)
 - Wftm/Lftm &ndash; Broj pogođenih slobodnih bacanja (1 poen)
 - Wfta/Lfta &ndash; Broj slobodnih bacanja (1 poen)
 - Wor/Lor &ndash; Broj uhvaćenih lopti nakon šuta u napadu
 - Wdr/Ldr &ndash; Broj uhvaćenih lopti nakon šuta u obrani
 - Wast/Last &ndash; Broj asistencija
 - Wto/Lto &ndash; Broj izgubljenih lopti u napadu
 - Wstl/Lstl &ndash; Broj oduzetih lopti
 - Wblk/Lblk &ndash; Broj blokiranja protivnika
 - Wpf/Lpf &ndash; Broj osobnih pogrešaka

In [54]:
rsdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60518 entries, 0 to 60517
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   season  60518 non-null  int64 
 1   daynum  60518 non-null  int64 
 2   wteam   60518 non-null  int64 
 3   wscore  60518 non-null  int64 
 4   lteam   60518 non-null  int64 
 5   lscore  60518 non-null  int64 
 6   wloc    60518 non-null  object
 7   numot   60518 non-null  int64 
 8   wfgm    60518 non-null  int64 
 9   wfga    60518 non-null  int64 
 10  wfgm3   60518 non-null  int64 
 11  wfga3   60518 non-null  int64 
 12  wftm    60518 non-null  int64 
 13  wfta    60518 non-null  int64 
 14  wor     60518 non-null  int64 
 15  wdr     60518 non-null  int64 
 16  wast    60518 non-null  int64 
 17  wto     60518 non-null  int64 
 18  wstl    60518 non-null  int64 
 19  wblk    60518 non-null  int64 
 20  wpf     60518 non-null  int64 
 21  lfgm    60518 non-null  int64 
 22  lfga    60518 non-null

In [55]:
rsdr.describe(include='all')

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
count,60518.000000,60518.000000,60518.000000,60518.000000,60518.000000,60518.000000,60518,60518.000000,60518.000000,60518.000000,...,60518.000000,60518.000000,60518.000000,60518.000000,60518.000000,60518.000000,60518.000000,60518.000000,60518.0000,60518.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,36172,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2008.683284,71.551737,1286.584851,74.583826,1282.465878,62.589973,NaN,0.071136,25.796275,54.596385,...,18.827770,12.126045,18.050084,11.430517,21.172329,11.427377,14.706517,6.157606,2.8591,19.852622
std,3.425290,35.159429,104.972198,11.054579,104.218602,10.835611,NaN,0.310614,4.678601,7.621741,...,5.763291,5.356468,7.151907,4.234023,4.469253,3.735019,4.495741,2.810648,2.0518,4.527769
min,2003.000000,0.000000,1101.000000,34.000000,1101.000000,20.000000,NaN,0.000000,10.000000,27.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.0000,5.000000
25%,2006.000000,40.000000,1196.000000,67.000000,1191.000000,55.000000,NaN,0.000000,23.000000,49.000000,...,15.000000,8.000000,13.000000,8.000000,18.000000,9.000000,12.000000,4.000000,1.0000,17.000000
50%,2009.000000,75.000000,1284.000000,74.000000,1281.000000,62.000000,NaN,0.000000,26.000000,54.000000,...,18.000000,12.000000,18.000000,11.000000,21.000000,11.000000,14.000000,6.000000,3.0000,20.000000
75%,2012.000000,102.000000,1379.000000,82.000000,1374.000000,69.000000,NaN,0.000000,29.000000,59.000000,...,22.000000,15.000000,23.000000,14.000000,24.000000,14.000000,17.000000,8.000000,4.0000,23.000000


#### Nedostajuće vrijednosti

In [69]:
rsdr.isna().sum()

season    0
daynum    0
wteam     0
wscore    0
lteam     0
lscore    0
wloc      0
numot     0
wfgm      0
wfga      0
wfgm3     0
wfga3     0
wftm      0
wfta      0
wor       0
wdr       0
wast      0
wto       0
wstl      0
wblk      0
wpf       0
lfgm      0
lfga      0
lfgm3     0
lfga3     0
lftm      0
lfta      0
lor       0
ldr       0
last      0
lto       0
lstl      0
lblk      0
lpf       0
dtype: int64

Možemo vidjeti da ova tablica nema nedostajuće vrijednosti

#### Rezultati svake utakmice bez detaljne statistike
Od sezone 1985. do 2014.

In [67]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(rscr)

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
134561,2014,132,1196,61,1246,60,N,0
134562,2014,132,1277,69,1276,55,N,0
134563,2014,132,1386,65,1433,61,N,0
134564,2014,132,1418,82,1209,81,N,1


In [65]:
rscr.describe(include='all')

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
count,134566.000000,134566.000000,134566.000000,134566.000000,134566.000000,134566.000000,134566,134566.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,H,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,80225,NaN
mean,2000.465140,75.573748,1286.570404,76.688614,1283.126169,64.562817,NaN,0.041638
std,8.680053,33.084248,104.564012,12.253340,104.976787,11.405004,NaN,0.239485
min,1985.000000,0.000000,1101.000000,34.000000,1101.000000,20.000000,NaN,0.000000
25%,1993.000000,47.000000,1197.000000,68.000000,1191.000000,57.000000,NaN,0.000000
50%,2001.000000,78.000000,1283.000000,76.000000,1281.000000,64.000000,NaN,0.000000
75%,2008.000000,103.000000,1379.000000,84.000000,1376.000000,72.000000,NaN,0.000000


#### Nedostajuće vrijednosti

In [70]:
rscr.isna().sum()

season    0
daynum    0
wteam     0
wscore    0
lteam     0
lscore    0
wloc      0
numot     0
dtype: int64

Možemo vidjeti da ova tablica nema nedostajuće vrijednosti

### Rezultati završnog turnira sezone
Rezulati svake utakmice u završnom turniru sezone sa potpunom statistikom od sezone 2003. do 2014.

In [56]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tdr)

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,wfgm3,wfga3,wftm,wfta,wor,wdr,wast,wto,wstl,wblk,wpf,lfgm,lfga,lfgm3,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2014,146,1163,60,1277,54,N,0,17,49,5,22,21,22,7,23,8,8,7,3,12,18,46,11,29,7,8,6,26,13,16,5,3,16
776,2014,146,1246,75,1276,72,N,0,31,58,7,11,6,11,17,18,8,11,2,6,14,27,57,7,18,11,14,14,10,11,7,4,1,14
777,2014,152,1163,63,1196,53,N,0,24,43,5,12,10,13,5,23,12,13,6,4,16,19,49,1,10,14,20,12,15,3,11,3,1,14
778,2014,152,1246,74,1458,73,N,0,29,58,2,5,14,21,11,21,9,4,4,1,17,23,50,8,20,19,20,6,21,11,8,4,3,18


Značajke su iste kao i u prethodnoj tablici za regularni dio sezone <br>

In [57]:
tdr.describe(include='all')

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
count,780.000000,780.000000,780.000000,780.000000,780.000000,780.000000,780,780.000000,780.000000,780.000000,...,780.000000,780.000000,780.000000,780.000000,780.000000,780.000000,780.000000,780.000000,780.000000,780.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,780,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2008.561538,139.087179,1291.653846,74.610256,1294.279487,63.188462,NaN,0.076923,25.985897,54.842308,...,19.770513,11.794872,17.000000,11.479487,20.982051,11.496154,12.471795,5.939744,2.917949,19.123077
std,3.465592,4.220810,102.276919,10.693268,106.652370,10.496487,NaN,0.311080,4.779724,7.190209,...,5.751812,5.123529,6.628797,4.209634,4.148982,3.658915,3.970103,2.759083,2.064042,4.247507
min,2003.000000,134.000000,1104.000000,47.000000,1102.000000,29.000000,NaN,0.000000,13.000000,34.000000,...,5.000000,0.000000,3.000000,1.000000,10.000000,2.000000,3.000000,0.000000,0.000000,7.000000
25%,2006.000000,136.000000,1210.000000,67.000000,1207.000000,56.000000,NaN,0.000000,23.000000,50.000000,...,16.000000,8.000000,12.000000,8.000000,18.000000,9.000000,10.000000,4.000000,1.000000,16.000000
50%,2009.000000,137.000000,1277.000000,74.000000,1293.000000,63.000000,NaN,0.000000,26.000000,54.500000,...,19.500000,11.000000,17.000000,11.000000,21.000000,12.000000,12.000000,6.000000,3.000000,19.000000
75%,2012.000000,139.000000,1393.000000,81.000000,1393.000000,70.000000,NaN,0.000000,29.000000,59.000000,...,23.000000,15.000000,21.000000,14.000000,24.000000,14.000000,15.000000,8.000000,4.000000,22.000000


#### Nedostajuće vrijednosti

In [71]:
tdr.isna().sum()

season    0
daynum    0
wteam     0
wscore    0
lteam     0
lscore    0
wloc      0
numot     0
wfgm      0
wfga      0
wfgm3     0
wfga3     0
wftm      0
wfta      0
wor       0
wdr       0
wast      0
wto       0
wstl      0
wblk      0
wpf       0
lfgm      0
lfga      0
lfgm3     0
lfga3     0
lftm      0
lfta      0
lor       0
ldr       0
last      0
lto       0
lstl      0
lblk      0
lpf       0
dtype: int64

Možemo vidjeti da ova tablica nema nedostajuće vrijednosti

#### Rezultati svake utakmice bez detaljne statistike
Od sezone 1985. do 2014.

In [68]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tcr)

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
1911,2014,146,1163,60,1277,54,N,0
1912,2014,146,1246,75,1276,72,N,0
1913,2014,152,1163,63,1196,53,N,0
1914,2014,152,1246,74,1458,73,N,0


In [72]:
tcr.describe(include='all')

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
count,1916.000000,1916.000000,1916.000000,1916.000000,1916.000000,1916.000000,1916,1916.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,1916,NaN
mean,1999.639875,139.171190,1284.301670,76.983820,1291.354906,65.263048,NaN,0.070981
std,8.693250,4.208151,100.211957,12.062641,103.013955,11.300441,NaN,0.287554
min,1985.000000,134.000000,1104.000000,43.000000,1102.000000,29.000000,NaN,0.000000
25%,1992.000000,136.000000,1207.000000,69.000000,1208.000000,58.000000,NaN,0.000000
50%,2000.000000,137.000000,1275.500000,76.000000,1285.500000,65.000000,NaN,0.000000
75%,2007.000000,139.000000,1381.250000,84.000000,1387.000000,72.000000,NaN,0.000000


#### Nedostajuće vrijednosti

In [73]:
tcr.isna().sum()

season    0
daynum    0
wteam     0
wscore    0
lteam     0
lscore    0
wloc      0
numot     0
dtype: int64

Možemo vidjeti da ova tablica nema nedostajuće vrijednosti

### Oznake timova na završnom turniru
Identifikacija oznaka za 64 tima u svakom zavšnom turniru

In [58]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tse)

,season,seed,team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
1941,2014,Z12,1295
1942,2014,Z13,1308
1943,2014,Z14,1418
1944,2014,Z15,1110


Pojašnjenja značajki:
 - Season &ndash; jedno-znakovni identifikator sezone
 - Seed &ndash; troznakovni oznaka tima
 - Team &ndash; Id momčadi

In [59]:
tse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   season  1946 non-null   int64 
 1   seed    1946 non-null   object
 2   team    1946 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 45.7+ KB


In [60]:
tse.describe(include='all')

,season,seed,team
count,1946.000000,1946,1946.000000
unique,NaN,86,NaN
top,NaN,W01,NaN
freq,NaN,30,NaN
mean,1999.637718,NaN,1290.664440
std,8.692685,NaN,102.916408
min,1985.000000,NaN,1102.000000
25%,1992.000000,NaN,1208.000000
50%,2000.000000,NaN,1285.000000
75%,2007.000000,NaN,1386.000000


### Turnirska uparivanja
Popis uparivanja utakmica u završnom turniru

In [61]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tsl)

,season,slot,strongseed,weakseed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12
...,...,...,...,...
1911,2014,R6CH,R5WX,R5YZ
1912,2014,X16,X16a,X16b
1913,2014,Y11,Y11a,Y11b
1914,2014,Y12,Y12a,Y12b


Pojašnjenja značajki:
 - Season &ndash; jedno-znakovni identifikator sezone
 - Slot &ndash; identifikator uparivanja timova u završnom turniru po rundama
 - Strongseed &ndash; oznaka momčadi ili pobjednika prošle runde za koje se smatra da će pobijediti u ovoj rundi
 - Weakseed &ndash; oznaka momčadi ili pobjednika prošle runde za koje se smatra da će izgubiti u ovoj rundi

In [62]:
tsl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   season      1916 non-null   int64 
 1   slot        1916 non-null   object
 2   strongseed  1916 non-null   object
 3   weakseed    1916 non-null   object
dtypes: int64(1), object(3)
memory usage: 60.0+ KB


In [63]:
tsl.describe(include='all')

,season,slot,strongseed,weakseed
count,1916.000000,1916,1916,1916
unique,NaN,74,74,74
top,NaN,R1W1,W01,W16
freq,NaN,30,30,30
mean,1999.639875,NaN,NaN,NaN
std,8.693250,NaN,NaN,NaN
min,1985.000000,NaN,NaN,NaN
25%,1992.000000,NaN,NaN,NaN
50%,2000.000000,NaN,NaN,NaN
75%,2007.000000,NaN,NaN,NaN
